# Reforumlated Markowitz Model - Optimization Codes 

Import the necessary libraries

In [36]:
import numpy as np
import pandas as pd
import cvxpy as cp  
import cvxopt
import random
import time

Importing the Data

In [2]:
df = pd.read_excel("Daily_Returns.xlsx")

Adding the date as an index

In [3]:
# Convert the 'date' column to datetime format 
df['Date'] = pd.to_datetime(df['Date'])

# Set the 'date' column as the index
df.set_index('Date', inplace=True)

Renaming the DataFrame

In [4]:
df_returns=df

Calculating the expected returns and the variance covariance matrix

In [5]:
# Calculate expected returns for each asset
expected_returns = df_returns.mean()

# Calculate the variance-covariance matrix
cov_matrix = df_returns.cov()


#Converting the expected returns to a numpy array
expected_returns = expected_returns.values

#The number of assets
n_assets = len(expected_returns)


Setting the rest of the constraints for the reforumated problem

In [22]:
# Maximum acceptable risk (R): The total risk that the portfolio can assume, based on variance-covariance matrix
R = 0.1

# Maximum units of each asset that can be purchased (D)
D = 5

# Maximum number of assets in the portfolio (K): The maximum number of different assets that can be included in the portfolio
K = 15

# The number of shares the investor would like to purchase
S = 20

In [23]:
# Variables to solve for
x = cp.Variable(n_assets, integer=True)  # Units of each asset to purchase
y = cp.Variable(n_assets, boolean=True)  # Binary variable for asset inclusion

In [24]:
# Creating the weights
w = x/S

Setting up the optimization problem

In [25]:

# Constraints
risk_constraint = cp.quad_form(w, cov_matrix) <=  R**2  # Risk constraint
diversification_constraints = [x[i] <= D*y[i] for i in range(n_assets)]  # Diversification and linking
cardinality_constraint = cp.sum(y) <= K  # Cardinality constraint
non_negativity_constraints = [x >= 0]  # Ensure x is non-negative 
total_sum_constraint = [cp.sum(x)==S]

# Objective function to maximize total expected return
objective = cp.Maximize(expected_returns @ w)

# Problem definition
problem = cp.Problem(objective, [risk_constraint, cardinality_constraint] + non_negativity_constraints +diversification_constraints + total_sum_constraint )

Solving the optimization problem

In [26]:
# Solving the problem
problem.solve(solver=cp.CPLEX)


# Results
print("Status:", problem.status)
print("Maximum Expected Return:", problem.value)
print("Units of each asset to purchase:", x.value)
print("Assets included in the portfolio:", y.value)

Status: optimal
Maximum Expected Return: 0.0012809926490892003
Units of each asset to purchase: [-0.  5. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  5. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0.  5. -0. -0. -0. -0. -0. -0. -0.  5. -0. -0. -0. -0. -0.
 -0.]
Assets included in the portfolio: [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

Now let us test to see if all the constraints are satisfied

In [33]:
# Convert x.value and y.value to numpy arrays for easy manipulation
x_optimized = np.array(x.value)
y_optimized = np.array(y.value)
w_optimized = np.array(x_optimized/S)

# Constraint 1: Risk constraint
risk_value = np.dot(np.dot(w_optimized.T, cov_matrix), w_optimized)
print(f"Risk Constraint Satisfied: {np.sqrt(risk_value) <= R}")
print(f"Risk Value: {np.sqrt(risk_value)} <= {R}")

# Constraint 2: Diversification and linking constraint
diversification_values = x_optimized <= D * y_optimized
print(f"Diversification and Linking Constraints Satisfied: {all(diversification_values)}")

# Constraint 3: Cardinality constraint
cardinality_value = (x_optimized > 0).sum()
print(f"Cardinality Constraint Satisfied: {cardinality_value <= K}")
print(f"Number of Assets Included: {cardinality_value} <= {K}")

# Constraint 4: Non-negativity constraint
non_negativity_satisfied = all(x_optimized >= 0)
print(f"Non-negativity Constraints Satisfied: {non_negativity_satisfied}") 

# Constraint 5: sum of shares constraint
total_share = np.sum(x_optimized)
print(f"Sum of units Constraint Satisfied: {total_share <= S}")
print(f"Number of Units Included: {total_share} <= {S}")

Risk Constraint Satisfied: True
Risk Value: 0.019487885441359105 <= 0.1
Diversification and Linking Constraints Satisfied: True
Cardinality Constraint Satisfied: True
Number of Assets Included: 4 <= 15
Non-negativity Constraints Satisfied: True
Sum of units Constraint Satisfied: True
Number of Units Included: 20.0 <= 20


Checking the assets to invest in

In [34]:
values_list = x.value

# Creating a new DataFrame with column names and values where values are > 0
filtered_data = {'Column Name': [], 'Value': []}
for column_name, value in zip(df.columns, values_list):
    if value > 0:
        filtered_data['Column Name'].append(column_name)
        filtered_data['Value'].append(value)

new_df = pd.DataFrame(filtered_data)

print(new_df)


  Column Name  Value
0        AAPL    5.0
1        ALGN    5.0
2        NFLX    5.0
3        TSLA    5.0


Looping it all

In [35]:
# Maximum acceptable risk (R): The total risk that the portfolio can assume, based on variance-covariance matrix
Variance_Values = [i / 100 for i in range(1, 21)]

#Sum of shares
S = 1000

In [37]:
"""

We will now loop over multiple risk values to calculate the return and get an efficient frontier and generate
each of the portfolios for these risk values. Note that all the constraints except the number of shares the investor should invest
in would be relaxed in this solution for the purpose of comparing with the vanilla markowitz model.

"""

results_list = []  # Initialize the list

for R in Variance_Values:

    
    # Check the variance value
    print(R)


    # Variables to solve for
    x = cp.Variable(n_assets, integer=True)  # Units of each asset to purchase
    y = cp.Variable(n_assets, boolean=True)  # Binary variable for asset inclusion

    # Creating the weights
    w = x/S

    # Constraints
    risk_constraint = cp.quad_form(w, cov_matrix) <=  R**2  # Risk constraint
    non_negativity_constraints = [x >= 0]  # Ensure x is non-negative 
    total_sum_constraint = [cp.sum(x)==S]


    # Objective function to maximize total expected return
    objective = cp.Maximize(expected_returns @ w)

    # Problem definition
    problem = cp.Problem(objective, [risk_constraint] + non_negativity_constraints + total_sum_constraint )

    # Time the sampler
    start_time = time.time()
        
    # Solving the problem
    problem.solve(solver=cp.CPLEX)

    time_taken = time.time() - start_time


    #Recreating the original variables
    x_optimized = np.array(x.value)
    y_optimized = np.array(y.value)
    w_optimized = x_optimized/S

 

    #Calculating the total return
    weights = [x/S for x in x.value]

    # Multiply corresponding elements and sum them up
    total = sum(x * y for x, y in zip(expected_returns, weights))

    # Constraint 1: Risk constraint
    risk_value = np.sqrt(np.dot(np.dot(w_optimized.T, cov_matrix), w_optimized))

    # Constraint 2: Non-negativity constraint
    non_negativity_satisfied = all(x_optimized >= 0)

    # Constraint 3: Sum of shares constraint
    sum_of_shares_satisfied = sum(x_optimized) == S

    #Creating the asset dataframe
    values_list = x_optimized.tolist()

    # Creating a new DataFrame with column names and values where values are > 0
    filtered_data = {'Asset': [], 'Value': []}
    for column_name, value in zip(df.columns, values_list):
        if value > 0:
            filtered_data['Asset'].append(column_name)
            filtered_data['Value'].append(value)

    # Create a dictionary for the current iteration
    iteration_results = {
        'time_taken': time_taken,
        'risk_value': risk_value,
        'R': R,
        'total_return': total,
        'risk_constraint_satisfied': risk_value <= R,
        'non_negativity_satisfied': non_negativity_satisfied,
        'Total_Shares_satisfied': sum_of_shares_satisfied,
        'assets_included_df': filtered_data
    }

    # Step 4: Append this dictionary to the list
    results_list.append(iteration_results)

0.01
0.02
0.03
0.04
0.05
0.06
0.07
0.08
0.09
0.1
0.11
0.12
0.13
0.14
0.15
0.16
0.17
0.18
0.19
0.2


Creating a dataframe of results

In [39]:
# Creating a DataFrame from results_list for specific metrics
metrics_columns = [
    'time_taken',
    'risk_value',
    'R',
    'total_return',
    'risk_constraint_satisfied',
    'non_negativity_satisfied',
    'Total_Shares_satisfied'
]

# Use a list comprehension to pick out these specific keys from each dictionary in the results list
final_results_data = [{col: result[col] for col in metrics_columns} for result in results_list]

# Convert this list of dictionaries to a DataFrame
final_results_df = pd.DataFrame(final_results_data)

# Save the DataFrame as an Excel file
final_results_df.to_excel('Final_Results.xlsx', index=False)


Creating the excel of asset allocations

In [40]:
for i, result in enumerate(results_list, start=1):
    # Convert filtered_data (which replaced new_df) to a DataFrame
    assets_df = pd.DataFrame(result['assets_included_df'])
    
    # Save the DataFrame as an Excel file
    file_name = f'Assets{i}.xlsx'
    assets_df.to_excel(file_name, index=False)